If you want to find out who are your best customers, using an old technique RFM matrix principle is still the best in the business. RFM means – Recency, Frequency and Monetary. RFM is basically a customer segmentation technique that works by using the past behaviour of your customers based on their purchases to split the customers into groups.

Let’s See How RFM Works:

As RFM is the technique behind customer segmentation, let’s see how it manages to do that. This technique is based on three main categories, they are:

- Recency(R) – Days since last purchase
- Frequency(F) – Total number of purchases
- Monetary Value(M) – How much total money the customer spent.

### Customer Segmentation with Python
Now, let’s proceed with the target of this PROJECT, which is to create a customer segmentation system with python. Now as I will use the RFM technique here, so the first thing we need to proceed is data because this technique is all dependent on data of customers expenditure on our products. The dataset I will use in this article can be downloaded below.

Now, let’s import the dataset and get started:

In [2]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
df = pd.read_excel(r"C:\Users\SHREE\Downloads\Python CODES\Customer Segmentation Using RFM\Online Retail.xlsx")
df.head()
df1 = df

The data contains all the transactions of a UK-based online retailer. All the transactions are between 01 Dec, 2010 and 09 Dec, 2011.

### Data Exploration
Now let’s explore the data by looking at –

- Missing values in important columns;
- Customers’ distribution in each country;
- Unit price and Quantity should > 0;
- Invoice date should < today.

In [3]:
df1.Country.nunique()

38

In [4]:
customer_country=df1[['Country','CustomerID']].drop_duplicates()
customer_country.groupby(['Country'])['CustomerID'].aggregate('count').reset_index().sort_values('CustomerID', ascending=False)

,Country,CustomerID
36,United Kingdom,3950
14,Germany,95
13,France,87
31,Spain,31
3,Belgium,25
33,Switzerland,21
27,Portugal,19
19,Italy,15
12,Finland,12
1,Austria,11


The above data states that more than 90 percent of the customers are from United Kingdom only. Some researchers states that customer segmentation varies with geography. So I will limit the data to United Kingdom only.

In [5]:
df1 = df1.loc[df1['Country'] == 'United Kingdom']

Now let’s check whether, there are null values in the dataset:

In [7]:
df1.isnull().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     133600
Country             0
dtype: int64

The above output stated that there are 133,600 missing values in the customerID column, and as we are working on customer segmentation, so we do not need to focus on customer ID, so I will simply remove these missing values in this column.

In [8]:
df1 = df1[pd.notnull(df1['CustomerID'])]

Now let’s remove the negative values in Quantity column:

In [10]:
df1 = df1[(df1['Quantity']>0)]

Now let’s add a new column for total price:

In [11]:
df1['TotalPrice'] = df1['Quantity'] * df1['UnitPrice']

To find out the first and last order dates in the data:

In [12]:
df1['InvoiceDate'].min()

Timestamp('2010-12-01 08:26:00')

In [13]:
df1['InvoiceDate'].max()

Timestamp('2011-12-09 12:49:00')

As recency is calculated for a point in time, and the last invoice date is 2011–12–09, so I will use 2011–12–10 to calculate recency:

In [14]:
import datetime as dt
NOW = dt.datetime(2011,12,10)
df1['InvoiceDate'] = pd.to_datetime(df1['InvoiceDate'])

### RFM Customer Segmentation
Now our data is completely ready for customer segmentation, lets move further with this by creating a RFM table:

In [15]:
rfmTable = df1.groupby('CustomerID').agg({'InvoiceDate': lambda x: (NOW - x.max()).days, 'InvoiceNo': lambda x: len(x), 'TotalPrice': lambda x: x.sum()})
rfmTable['InvoiceDate'] = rfmTable['InvoiceDate'].astype(int)
rfmTable.rename(columns={'InvoiceDate': 'recency', 
                         'InvoiceNo': 'frequency', 
                         'TotalPrice': 'monetary_value'}, inplace=True)

To calculate RFM metrics for each customer:

In [16]:
rfmTable.head()

,recency,frequency,monetary_value
CustomerID,,,
12346.0,325,1,77183.60
12747.0,2,103,4196.01
12748.0,0,4596,33719.73
12749.0,3,199,4090.88
12820.0,3,59,942.34


### Observations:

- CustomerID 12346 has frequency: 1, monetary value: 77,183.60 and recency: 325 days.
- CustomerID 12747 has frequency: 103, monetary value: 4,196.01 and recency: 2 days

### Split the metrics
The easiest way to split metrics into segments is by using quartiles.

- This gives us a starting point for the detailed analysis.
- 4 segments are easy to understand and explain.

In [17]:
quantiles = rfmTable.quantile(q=[0.25,0.5,0.75])
quantiles = quantiles.to_dict()

### Create a Customer Segment RFM table

In [18]:
segmented_rfm = rfmTable

The lowest recency, highest frequency and best monetary amounts will convey are our best customers:

In [19]:
def RScore(x,p,d):
    if x <= d[p][0.25]:
        return 1
    elif x <= d[p][0.50]:
        return 2
    elif x <= d[p][0.75]: 
        return 3
    else:
        return 4
    
def FMScore(x,p,d):
    if x <= d[p][0.25]:
        return 4
    elif x <= d[p][0.50]:
        return 3
    elif x <= d[p][0.75]: 
        return 2
    else:
        return 1

### Add segment numbers

In [20]:
segmented_rfm['r_quartile'] = segmented_rfm['recency'].apply(RScore, args=('recency',quantiles,))
segmented_rfm['f_quartile'] = segmented_rfm['frequency'].apply(FMScore, args=('frequency',quantiles,))
segmented_rfm['m_quartile'] = segmented_rfm['monetary_value'].apply(FMScore, args=('monetary_value',quantiles,))
segmented_rfm.head()

,recency,frequency,monetary_value,r_quartile,f_quartile,m_quartile
CustomerID,,,,,,
12346.0,325,1,77183.60,4,4,1
12747.0,2,103,4196.01,1,1,1
12748.0,0,4596,33719.73,1,1,1
12749.0,3,199,4090.88,1,1,1
12820.0,3,59,942.34,1,2,2


RFM segments split the customer base into an imaginary 3D cube which is hard to visualize. However, we can sort it out:

In [22]:
segmented_rfm['RFMScore'] = segmented_rfm.r_quartile.map(str) + segmented_rfm.f_quartile.map(str) + segmented_rfm.m_quartile.map(str)
segmented_rfm.head()

,recency,frequency,monetary_value,r_quartile,f_quartile,m_quartile,RFMScore
CustomerID,,,,,,,
12346.0,325,1,77183.60,4,4,1,441
12747.0,2,103,4196.01,1,1,1,111
12748.0,0,4596,33719.73,1,1,1,111
12749.0,3,199,4090.88,1,1,1,111
12820.0,3,59,942.34,1,2,2,122


### Let’s See The Top 10 of best customers

In [23]:
segmented_rfm[segmented_rfm['RFMScore']=='111'].sort_values('monetary_value', ascending=False).head(10)

,recency,frequency,monetary_value,r_quartile,f_quartile,m_quartile,RFMScore
CustomerID,,,,,,,
18102.0,0,431,259657.30,1,1,1,111
17450.0,8,337,194550.79,1,1,1,111
17511.0,2,963,91062.38,1,1,1,111
16684.0,4,277,66653.56,1,1,1,111
14096.0,4,5111,65164.79,1,1,1,111
13694.0,3,568,65039.62,1,1,1,111
15311.0,0,2379,60767.90,1,1,1,111
13089.0,2,1818,58825.83,1,1,1,111
15769.0,7,130,56252.72,1,1,1,111
